# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [1]:
# Imports
import os
import json
import sqlite3
import requests
from datetime import datetime
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr


In [2]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
openweather_api_key = os.getenv('OPENWEATHER_API_KEY')
exchangerate_api_key = os.getenv('EXCHANGERATE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key found: {openai_api_key[:8]}...")
else:
    print("OpenAI API Key not set")

if openweather_api_key:
    print(f"OpenWeather API Key found: {openweather_api_key[:5]}...")
else:
    print("OpenWeather API Key not set - Get one at https://openweathermap.org/api")

if exchangerate_api_key:
    print(f"ExchangeRate API Key found: {exchangerate_api_key[:5]}...")
else:
    print("ExchangeRate API Key not set - Get one at https://www.exchangerate-api.com/")


OpenAI API Key found: sk-proj-...
OpenWeather API Key found: c2fcd...
ExchangeRate API Key found: ce0f6...


In [3]:
openai = OpenAI()

ollama_url = "http://localhost:11434/v1"
ollama = OpenAI(api_key="ollama", base_url=ollama_url)

# Test if Ollama is available
ollama_available = False
test_response = requests.get("http://localhost:11434/", timeout=2)

if test_response.status_code == 200:
    ollama_available = True
    print("Ollama is running - Llama model available")
else:
    print("Ollama is not responding - Only GPT will be available")


DB = "bookings.db"

MODELS = {
    "GPT": "gpt-4.1-mini",
    "Llama": "llama3.2"
}


Ollama is running - Llama model available


In [4]:
# Create database and tables
def setup_database():
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        
        # Create prices table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS prices (
                city TEXT PRIMARY KEY,
                price REAL
            )
        ''')
        
        # Create bookings table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS bookings (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                city TEXT,
                passenger_name TEXT,
                travel_date TEXT,
                booking_date TEXT,
                status TEXT
            )
        ''')
        
        conn.commit()
    print("Database created successfully")

# Populate sample ticket prices
def populate_sample_data():
    ticket_prices = {
        "london": 799,
        "paris": 899,
        "tokyo": 1420,
        "sydney": 2999,
        "berlin": 499,
        "rome": 650,
        "new york": 450,
        "dubai": 1200,
        "singapore": 1350,
        "barcelona": 720
    }
    
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        for city, price in ticket_prices.items():
            cursor.execute(
                'INSERT OR REPLACE INTO prices (city, price) VALUES (?, ?)',
                (city.lower(), price)
            )
        conn.commit()
    print(f"Added {len(ticket_prices)} sample ticket prices")

# Run setup
setup_database()
populate_sample_data()


Database created successfully
Added 10 sample ticket prices


In [15]:
system_message = """
You are a helpful assistant for FlightAI Airlines.
You can help customers with:
- Checking ticket prices
- Booking flights
- Checking booking status
- Getting destination information
- Checking weather forecasts
- Converting currency

Always be courteous and professional.
If you need to use a tool, use it to provide accurate information.
When booking tickets, always confirm the details with the customer.
Your first response should be a greeting and a brief introduction of what you can do.
"""


In [6]:
def get_ticket_price(city):
    """Get the price of a ticket to the specified city"""
    print(f"Tool called: get_ticket_price({city})")
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT price FROM prices WHERE city = ?', (city.lower(),))
        result = cursor.fetchone()
        if result:
            return f"A ticket to {city.title()} costs ${result[0]}"
        else:
            return f"Sorry, we don't have pricing information for {city.title()} at this time."

def book_ticket(city, passenger_name, travel_date):
    """Book a ticket for a passenger"""
    print(f"Tool called: book_ticket({city}, {passenger_name}, {travel_date})")
    
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT price FROM prices WHERE city = ?', (city.lower(),))
        price_result = cursor.fetchone()
        
        if not price_result:
            return f"Sorry, we don't fly to {city.title()} at this time."
        
        # Create booking
        booking_date = datetime.now().strftime('%Y-%m-%d')
        cursor.execute(
            'INSERT INTO bookings (city, passenger_name, travel_date, booking_date, status) VALUES (?, ?, ?, ?, ?)',
            (city.lower(), passenger_name, travel_date, booking_date, 'confirmed')
        )
        booking_id = cursor.lastrowid
        conn.commit()
        
        return f"Booking confirmed! Booking ID: {booking_id}. Passenger: {passenger_name}, Destination: {city.title()}, Travel Date: {travel_date}, Price: ${price_result[0]}"

def get_booking_status(booking_id):
    """Check the status of a booking"""
    print(f"Tool called: get_booking_status({booking_id})")
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute(
            'SELECT id, city, passenger_name, travel_date, booking_date, status FROM bookings WHERE id = ?',
            (booking_id,)
        )
        result = cursor.fetchone()
        
        if result:
            return f"Booking #{result[0]} - Passenger: {result[2]}, Destination: {result[1].title()}, Travel Date: {result[3]}, Status: {result[5].upper()}"
        else:
            return f"No booking found with ID {booking_id}"


In [7]:
def get_destination_info(city):
    """Get information about a destination using REST Countries API"""
    print(f"Tool called: get_destination_info({city})")
    
    # Map common cities to countries
    city_to_country = {
        "london": "United Kingdom",
        "paris": "France",
        "tokyo": "Japan",
        "sydney": "Australia",
        "berlin": "Germany",
        "rome": "Italy",
        "new york": "United States",
        "dubai": "United Arab Emirates",
        "singapore": "Singapore",
        "barcelona": "Spain"
    }
    
    country = city_to_country.get(city.lower())
    if not country:
        return f"Sorry, I don't have detailed information about {city.title()} at this time."
    
    try:
        response = requests.get(f"https://restcountries.com/v3.1/name/{country}", timeout=5)
        if response.status_code == 200:
            data = response.json()[0]
            name = data.get('name', {}).get('common', country)
            capital = data.get('capital', ['N/A'])[0]
            region = data.get('region', 'N/A')
            languages = ', '.join(data.get('languages', {}).values())
            currency_info = data.get('currencies', {})
            currency = list(currency_info.keys())[0] if currency_info else 'N/A'
            currency_name = currency_info[currency].get('name', 'N/A') if currency_info else 'N/A'
            timezone = data.get('timezones', ['N/A'])[0]
            
            return f"""{city.title()} is in {name}. 
                        Capital: {capital}
                        Region: {region}
                        Languages: {languages}
                        Currency: {currency_name} ({currency})
                        Timezone: {timezone}"""
        else:
            return f"Unable to retrieve information about {city.title()}"
    except Exception as e:
        return f"Error fetching destination info: {str(e)}"

def get_weather_forecast(city):
    """Get weather forecast for a destination using OpenWeatherMap API"""
    print(f"🔧 Tool called: get_weather_forecast({city})")
    
    if not openweather_api_key:
        return "Weather service unavailable. Please set OPENWEATHER_API_KEY in your .env file."
    
    try:
        # Get coordinates first
        geo_url = f"http://api.openweathermap.org/geo/1.0/direct?q={city}&limit=1&appid={openweather_api_key}"
        geo_response = requests.get(geo_url, timeout=5)
        
        if geo_response.status_code == 200 and geo_response.json():
            geo_data = geo_response.json()[0]
            lat, lon = geo_data['lat'], geo_data['lon']
            
            # Get weather forecast
            weather_url = f"http://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={openweather_api_key}&units=metric"
            weather_response = requests.get(weather_url, timeout=5)
            
            if weather_response.status_code == 200:
                weather_data = weather_response.json()
                forecasts = weather_data['list'][:5]  # Next 5 forecasts (15 hours)
                
                forecast_text = f"Weather forecast for {city.title()}:\n"
                for forecast in forecasts:
                    time = forecast['dt_txt']
                    temp = forecast['main']['temp']
                    description = forecast['weather'][0]['description']
                    forecast_text += f"\n{time}: {temp}°C, {description}"
                
                return forecast_text
        
        return f"Unable to retrieve weather forecast for {city.title()}"
    except Exception as e:
        return f"Error fetching weather: {str(e)}"

def convert_currency(amount, from_currency, to_currency):
    """Convert currency using ExchangeRate API"""
    print(f"Tool called: convert_currency({amount} {from_currency} to {to_currency})")
    
    if not exchangerate_api_key:
        return "Currency conversion unavailable. Please set EXCHANGERATE_API_KEY in your .env file."
    
    try:
        url = f"https://v6.exchangerate-api.com/v6/{exchangerate_api_key}/pair/{from_currency.upper()}/{to_currency.upper()}/{amount}"
        response = requests.get(url, timeout=5)
        
        if response.status_code == 200:
            data = response.json()
            if data.get('result') == 'success':
                converted = data['conversion_result']
                rate = data['conversion_rate']
                return f"{amount} {from_currency.upper()} = {converted:.2f} {to_currency.upper()} (Rate: {rate:.4f})"
        
        return "Unable to convert currency. Please check the currency codes."
    except Exception as e:
        return f"Error converting currency: {str(e)}"


Tools Calling For OPEN AI function call

In [8]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_ticket_price",
            "description": "Get the price of a return ticket to a destination city",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The destination city"
                    }
                },
                "required": ["city"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "book_ticket",
            "description": "Book a flight ticket for a passenger",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The destination city"
                    },
                    "passenger_name": {
                        "type": "string",
                        "description": "The full name of the passenger"
                    },
                    "travel_date": {
                        "type": "string",
                        "description": "The travel date in YYYY-MM-DD format"
                    }
                },
                "required": ["city", "passenger_name", "travel_date"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_booking_status",
            "description": "Check the status of a booking using the booking ID",
            "parameters": {
                "type": "object",
                "properties": {
                    "booking_id": {
                        "type": "integer",
                        "description": "The booking ID number"
                    }
                },
                "required": ["booking_id"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_destination_info",
            "description": "Get detailed information about a destination city including country, capital, language, currency, and timezone",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The destination city"
                    }
                },
                "required": ["city"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_weather_forecast",
            "description": "Get the weather forecast for a destination city",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The destination city"
                    }
                },
                "required": ["city"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "convert_currency",
            "description": "Convert an amount from one currency to another",
            "parameters": {
                "type": "object",
                "properties": {
                    "amount": {
                        "type": "number",
                        "description": "The amount to convert"
                    },
                    "from_currency": {
                        "type": "string",
                        "description": "The source currency code (e.g., USD, EUR, GBP)"
                    },
                    "to_currency": {
                        "type": "string",
                        "description": "The target currency code (e.g., USD, EUR, GBP)"
                    }
                },
                "required": ["amount", "from_currency", "to_currency"],
                "additionalProperties": False
            }
        }
    }
]

print(f"Defined {len(tools)} tools for the assistant")


Defined 6 tools for the assistant


Tool Handler

In [9]:
def handle_tool_calls(message):
    """Process all tool calls in a message and return responses"""
    responses = []
    
    for tool_call in message.tool_calls:
        function_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        
        # Call the appropriate function
        if function_name == "get_ticket_price":
            result = get_ticket_price(arguments.get('city'))
        elif function_name == "book_ticket":
            result = book_ticket(
                arguments.get('city'),
                arguments.get('passenger_name'),
                arguments.get('travel_date')
            )
        elif function_name == "get_booking_status":
            result = get_booking_status(arguments.get('booking_id'))
        elif function_name == "get_destination_info":
            result = get_destination_info(arguments.get('city'))
        elif function_name == "get_weather_forecast":
            result = get_weather_forecast(arguments.get('city'))
        elif function_name == "convert_currency":
            result = convert_currency(
                arguments.get('amount'),
                arguments.get('from_currency'),
                arguments.get('to_currency')
            )
        else:
            result = f"Unknown function: {function_name}"
        
        responses.append({
            "role": "tool",
            "content": result,
            "tool_call_id": tool_call.id
        })
    
    return responses


Chat Function wuth streaming

In [10]:
def chat(message, history, model_choice):
    """Main chat function with streaming and tool support"""
    
    # Select the appropriate client
    if model_choice == "Llama" and not ollama_available:
        yield "Llama is not available. Please start Ollama with 'ollama serve' or select GPT."
        return
    
    client = ollama if model_choice == "Llama" else openai
    model = MODELS[model_choice]
    
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools
    )
    
    # Handle tool calls in a loop
    while response.choices[0].finish_reason == "tool_calls":
        assistant_message = response.choices[0].message
        tool_responses = handle_tool_calls(assistant_message)
        
        messages.append(assistant_message)
        messages.extend(tool_responses)
        
        # Get next response
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools
        )
    
    # Get final response with streaming
    result = response.choices[0].message.content or ""
    
    # Stream the result
    stream = client.chat.completions.create(
        model=model,
        messages=messages + [{"role": "assistant", "content": result}][:-1],
        stream=True
    )
    
    streamed_response = ""
    for chunk in stream:
        streamed_response += chunk.choices[0].delta.content or ''
        yield streamed_response


In [13]:
# Define this variable and then pass js=force_dark_mode when creating the Interface

force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""

In [ ]:
# Create the Gradio interface
with gr.Blocks(title="FlightAI Assistant", js=force_dark_mode) as demo:
    gr.Markdown("""
    # ✈️ FlightAI Assistant
    
    Welcome to FlightAI! I can help you with:
    Checking ticket prices, Booking flights, Checking booking status, Getting destination information
    Checking weather forecasts, Converting currencies

    """)
    
    with gr.Row():
        model_selector = gr.Dropdown(
            choices=["GPT", "Llama"],
            value="GPT",
            label="Select Model",
            info="Choose between OpenAI GPT or local Llama (via Ollama)"
        )
    
    chatbot = gr.ChatInterface(
        fn=chat,
        additional_inputs=[model_selector],
        type="messages",
        examples=[
            ["What's the price to London?", "GPT"],
            ["Tell me about Paris", "GPT"],
            ["What's the weather like in Tokyo?", "Llama"],
            ["Convert 799 USD to EUR", "GPT"],
            ["Book me a ticket to Berlin for John Smith on 2024-12-15", "GPT"],
            ["Check booking status for booking ID 1", "Llama"]
        ],
        title=None,
        description=None
    )

demo.launch()


* Running on local URL:  http://127.0.0.1:7879
* To create a public link, set `share=True` in `launch()`.


Tool called: get_ticket_price(London)
Tool called: get_destination_info(London)
Tool called: convert_currency(799 USD to NGN)
